<a href="https://colab.research.google.com/github/mahsaShv/Fraud-Detection-/blob/main/Rahnema_College_Fraud_Detection_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

دیتا را لود میکنیم

In [ ]:
import pandas as pd
file = open("/dataset.csv")
dataset = file.read()
rows = dataset.split("\n")
flag = True
properties = None
dictionary = []
for line in rows:
  if (flag):
    flag = False
    properties = line.split(",")
    continue
  dictionary.append(line.split(","))
df = pd.DataFrame(dictionary, columns = properties)
df




,id,ProId,ProCreationTimestamp,CustId,CustCreationTimestamp,Rate,TaskCreationTimestamp,Price,ServiceId,CategoryId,ZoneId,TaskState,TargetProId,isFraud
0,VkSa32MyS738HMkfk4tEfk,vqUkxUDuEmB7gHWQvcYrBn,1552751751302,SgcxJApdnttAvEN9865TSZ,1565441404199,5,1565608270089,362000,bKv8JeCDhHBCe8SnfDcv66,VzkJEPR5H3mnqctJ8TW3YS,98DpwFpuLYJtF6jajghriE,Finished,vqUkxUDuEmB7gHWQvcYrBn,FALSE
1,FDPHLhrcjx3Gww4syg43Po,KaLEhwzZxCQ7GjPmVwBVav,1558262535289,EveEDVPDDDq3Ljh3bg8Exn,1561655121208,5,1563280381472,,aegfi8HeWASAP6a9YkddeL,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,YvvjyjpmXxj43RJGBHPq2n,FALSE
2,6Z7CCQoHpC8ga8KZbsgUbx,pybDYgTiCUv3Pv3WLgxKCM,1558538960723,mrDLWRmWuxGD7TVPAsD5X6,1545554406049,1,1566297337468,,uDHVz7XXXRCkGKWRUe2jKp,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,FALSE
3,n5DKFF8SJYoP57ypvpCEg5,zqPiDV33KwrMBZoyeQXMJW,1507727811649,aK3MnLeQywfLYMJJNyu7i8,1512288789660,4,1522825078700,,Jy7nZvUqvwDUhcAG59wPcg,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,FALSE
4,oCwEhmYhhzHkuZyqQCusFU,CR8z3ThPyzBKXFqqzemQAS,1560860808052,vHjzucgXFu9eA9YhuEeuk4,1561328838459,1,1564542737897,150000,f8o6yBmfeJiUTAmvpRCFAZ,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24850,PjrU9WNm9qUqckqKgC93VP,qRGniU8hJzZQ2sWA3TvMTY,1545460832847,mWAu9Ni5QKSGnUF3YRzh8N,1556440396939,5,1556506495422,45000,aAkNmQxRakik386nZvYv9d,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,ZteCh5bw6tqeQ4MssMNA3f,FALSE
24851,KDPdrm7ewtvA9B8KUxYNwR,zeKLDm8J3EtxjQNHdxxo8J,1561457503776,VSvzqwmjUANsWvLMQummYi,1545058611086,1,1574769932447,,VKoejhc3preQHi3wrVZKKP,JdqYJwUosvwivsFdThmcXe,XZCnwH2vCRhHReoVx9X3XV,Finished,,FALSE
24852,MPBuJWnPpdGWn6hD9Q47Q8,FDyGBLqZe8WMmSbbiQYyMJ,1516171856542,NdpNin2R8KoizHHY67MYwx,1525990442355,5,1526195732908,100000,yHdwNG6WZx4N8nPdRJqzND,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,FALSE
24853,wH4QMZpHK2FWBddjeEqxFV,vTYkudyaVUfPfdKhWy8tRk,1513248833327,Z9ff8Aq82mKUwcrQdchgAJ,1551456541794,5,1551543231278,600000,2wj2Bn7pJzF2bMogvaSbj6,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,kMLhispLCzyXTR8b8g6ujk,FALSE


In [ ]:
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import numpy as np
df.isna().sum()

id                       0
ProId                    0
ProCreationTimestamp     0
CustId                   0
CustCreationTimestamp    0
Rate                     0
TaskCreationTimestamp    0
Price                    0
ServiceId                0
CategoryId               0
ZoneId                   0
TaskState                0
TargetProId              0
isFraud                  0
dtype: int64

In [ ]:
df['Price'] = pd.to_numeric(df['Price'])
df['Rate'] = pd.to_numeric(df['Rate'])
#df['CustCreationTimestamp'] = pd.to_numeric(df['CustCreationTimestamp'])
#df['TaskCreationTimestamp'] = pd.to_numeric(df['TaskCreationTimestamp'])
df['isFraud' ] = df['isFraud'].replace('FALSE',0)
df['isFraud' ] = df['isFraud'].replace('TRUE',1)
df['Price'].fillna(df['Price'].mean(), inplace=True)
df['Rate'].fillna(df['Rate'].mean(), inplace=True)
df = pd.get_dummies(df, columns=['Rate'], drop_first=True)
df['idChoseProid'] = np.where(df["ProId"] == df["TargetProId"], 1, 0)
df['TaskSubCust'] = (pd.to_datetime(df['TaskCreationTimestamp'], unit='ms') - pd.to_datetime(df['CustCreationTimestamp'], unit='ms')).dt.days
dataFrame, testDF = train_test_split(df)
trainDFwithFraud = dataFrame[dataFrame['isFraud'] == 1]
dataFrame = pd.concat([dataFrame, trainDFwithFraud,trainDFwithFraud,trainDFwithFraud,trainDFwithFraud,trainDFwithFraud,trainDFwithFraud])
dataFrame.shape, testDF.shape


((25553, 23), (6214, 23))

In [ ]:
dataFrame

,id,ProId,ProCreationTimestamp,CustId,CustCreationTimestamp,TaskCreationTimestamp,Price,ServiceId,CategoryId,ZoneId,TaskState,TargetProId,isFraud,Rate_-2,Rate_-1,Rate_0,Rate_1,Rate_2,Rate_3,Rate_4,Rate_5,idChoseProid,TaskSubCust
3666,MzeYnKsSK2q8ScGLUfHGhZ,82LCndDykQAJDt5o5LNjwr,1512392853331,qDgJh2sERfQczJLtUDyCSM,1548578218620,1548585586676,2.500000e+05,Sosm2uC8zEpAHt4HRzEQSU,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,0
19461,r3wUCjDcTbRPq6nFknwJBs,3ModGDrYhnA86E98nhwrnh,1549788455901,JPV63vpreSViNQiSaeCvMN,1556512003686,1556867677328,2.000000e+05,tsAWT4mUsYMkdFoyqpTJTX,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,sH34gauJfYQxPeAKDsMR8j,0,0,0,0,0,0,0,0,1,0,4
21419,J4APYusP5sMnRuuZvrxxhM,t4irRhgP3HCDnvEh9khe8Q,1540067378335,iuqUsQkaFsunvBGhzgaiZw,1541255015361,1541396248799,9.000000e+04,mMTCfv4EXt6D74XiKxf33G,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,1
5513,k53aKfNBBxWcfLkJqRmjPq,QX9VMUPeHKbb8Dy6djGecc,1555690425963,TXDvjbM9rY8DWA8CEPaahk,1570667241483,1570886024439,5.137638e+05,UggUAaDGeoWLnK8EVtxHRr,JdqYJwUosvwivsFdThmcXe,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,1,0,0,0,0,0,2
23136,oGF9766xqnDyTQNwgTFUde,zEKMcqaT79z6xyk23Z6xbx,1518105424286,itBWZKk7xwCRuJ82BB7GZu,1518701710318,1519654989869,8.000000e+04,Jy7nZvUqvwDUhcAG59wPcg,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17740,RJfqaMazuswaPJMdatpTqE,j2nx97Pex3uzMHv2kEF8GJ,1529051205265,hGhQLKXeq753nvQtowiumm,1556546457172,1556620336041,1.100000e+05,acmdm2pESDGqd3bL9dVhrN,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,1,0,0,0,0,0,0,0,1,0,0
16050,DCkhFwCbFzRC2uYk8hKkQ6,piTYfvXVuSVYwtJUjzN2k9,1545728130028,9xcLh6NF5gqxwn2pisY2oi,1547203437987,1562040808466,5.137638e+05,5kmdrs3BfWRugmzB65Mkjf,Ax5bRJZGir8vaZMs3ZB9i5,XZCnwH2vCRhHReoVx9X3XV,Finished,piTYfvXVuSVYwtJUjzN2k9,1,0,0,0,0,0,0,0,1,1,171
10475,bGqzcXmT3otzd3ffGdk934,qv8L6yefk2guJzZn4FtGMj,1528991267906,Q84GtkA76YUho6KbLuQJb9,1549008031191,1569326705743,9.000000e+04,UX7m6RWnKWvmw82URCyUMB,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,1,0,0,0,0,0,0,0,1,0,235
19633,R5LtZXtCxL4gWUXdQBde9e,otB3LBPMcK9LjuoasJvWV9,1534357455387,NFm5n5nvbLR6ANh5n5qz5s,1546716719220,1546716908666,5.000000e+06,q4CYPYjkJs7zuxj3z2HRwt,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
col=dataFrame[dataFrame['isFraud'] == 1].groupby(['ProId'])['id'].count()
col

ProId
258pjAdXRonen93HG5M4ps    21
25nfdbp92SZceT4wG6ijxf     7
2BpSXsj9QcWPCiq8sDBQhD    14
2GpNp9GWeSGsALch6H6jUM    21
2SeuQZKiaDUgHw9jXuosc4     7
                          ..
zST7cmXcoSjZTz3Fe7Vp3r    21
zTzL8hh3Acp6oMJ5c3FXRa     7
zU666ugjdaX6PKe877YZaA     7
zcZzfYK3rXP6NLQMmxG9wM    14
zuXcrV2GnoMzh8igxqZiRj    14
Name: id, Length: 743, dtype: int64

In [ ]:
col=testDF[testDF['isFraud'] == 1].groupby(['ProId'])['id'].count()
col

ProId
28D7k64dwp7H5cLSPjzHo4    3
2BGiVSvvgrCnLPceJ7fTGY    1
2GpNp9GWeSGsALch6H6jUM    1
2kSChFGL7cjFDjFENkih7L    1
2mUdmhmsDRdSJVGf5WrJxB    1
                         ..
ybFkNxcqqgHQP5dRXUHB7C    1
yew7jEuE8CYdTJXstwBnpP    1
z4a3ebUEnRgCU3aZozAEPz    1
zV7r6iJud7T3D6Wb5tPLQF    1
ze6TdKm2HLftoMnNLiwTmU    1
Name: id, Length: 318, dtype: int64

In [ ]:
colT = pd.DataFrame([col.values]).T
colT['ProId'] = col.keys()
dataFrame['Count'] = dataFrame.join(colT.set_index('ProId'), on='ProId')[0]
testDF['Count'] = testDF.join(colT.set_index('ProId'), on='ProId')[0]
dataFrame['Count' ] = dataFrame['Count' ].replace('NaN',0)
testDF['Count' ] = testDF['Count' ].replace('NaN',0)
dataFrame['Count'].fillna(0,inplace=True)
testDF['Count'].fillna(0,inplace=True)
testDF
#col.keys()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,id,ProId,ProCreationTimestamp,CustId,CustCreationTimestamp,TaskCreationTimestamp,Price,ServiceId,CategoryId,ZoneId,TaskState,TargetProId,isFraud,Rate_-2,Rate_-1,Rate_0,Rate_1,Rate_2,Rate_3,Rate_4,Rate_5,idChoseProid,TaskSubCust,Count
17370,LiKvDmVP3vfgHAQfNB9imM,YW7WSmGCWUHwiNKxNGNE9B,1542221202672,Su24TxLze7STfiknqfdDNE,1534486545557,1542712941885,3.000000e+06,XmvWkiYoHtwefiSTQNug9H,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,95,0.0
20868,QdTJYMj3mwkY7aB6gQENYB,N7gMBSBKk8FgBZyPHvs36k,1546188882496,59tdDiLGQkSKyrVAbkScBT,1564123581236,1569672674662,5.137638e+05,5YkLrEdys7QnTpdrsj7QKT,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,1,0,0,64,1.0
1904,7JmahzhvfQmtTVz2GEe7bx,L3hhCtVpGKYpKnUHrPaqav,1531063486372,x9ZgjwqzTTuRZ3VqhdBaSw,1559997303824,1561214218373,8.000000e+04,aAkNmQxRakik386nZvYv9d,VzkJEPR5H3mnqctJ8TW3YS,Exa7SPbiAcpe6Px23obGiX,Finished,27pDraJohuj3EdA2nLSduo,0,0,0,0,0,0,0,0,1,0,14,2.0
9542,pdxcMppEJpCoBWL2vJu4Vn,dkwSgXTyPhgfX7xQvq9PTm,1546670392940,hjFHqfqcpnLaXhbaAUVpTQ,1535987981665,1555919715987,5.000000e+05,Enkg2t2yLQmEPTbRVCQNBE,JdqYJwUosvwivsFdThmcXe,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,230,0.0
23656,JTQGGaDN8UPCLiynvtBEVw,v28oetHB4MzgEAgMqSF8wq,1572939934031,HqEoQdF6vLyLeBWwrFAP6h,1575462173266,1575478286760,4.000000e+05,LossceDVhNXShW4PKvnmQJ,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20088,cYBAUHogi678EsEKhnBLmV,u2ttxyNsewmgNN4CUeaXtC,1520925023793,EQrtaEpMegvyKK5DWsibkX,1520834912552,1524892921390,9.000000e+04,UX7m6RWnKWvmw82URCyUMB,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,46,1.0
23332,MXBJYetvMWipkWYiwUGfr3,txiiLms9BgpLWzt6jGJsne,1507727811649,QkCiFU87uvT5x6SW8MDqiu,1504015258011,1504015258011,5.137638e+05,UX7m6RWnKWvmw82URCyUMB,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,0,1.0
19422,azNaEUSiwf4eyJXfDSKZak,PEci9VmKkFhWdx5JQC7UPV,1507727811649,sKWzGR8yr3VgmmKzy4YdDF,1532885362214,1536749938197,5.137638e+05,UX7m6RWnKWvmw82URCyUMB,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,1,0,0,0,0,0,44,4.0
18989,L6RZo7uQrRvKbLX2fuPuSS,wnDfDpUX6J6bC8gLimVHiS,1515853712972,r2X2Eqf8Fa9MAT4ahJrJet,1527500247419,1527569973582,5.000000e+04,aAkNmQxRakik386nZvYv9d,VzkJEPR5H3mnqctJ8TW3YS,XZCnwH2vCRhHReoVx9X3XV,Finished,,0,0,0,0,0,0,0,0,1,0,0,0.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
lr =  RandomForestClassifier()
from sklearn.model_selection import cross_val_predict, cross_val_score
train_cols = ['Rate_0','Rate_1','Rate_2','Rate_3','Rate_4','Rate_5','idChoseProid','TaskSubCust','Count']
test_col = 'isFraud'
cross_val_score(lr, dataFrame[train_cols], dataFrame[test_col], cv=5)
lr.fit(dataFrame[train_cols], dataFrame[test_col])
probs = lr.predict_proba(dataFrame[train_cols])
psur = probs[:,1]
preds = lr.predict(dataFrame[train_cols])
preds

array([1, 0, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score
#precision_score(dataFrame['isFraud'], preds), recall_score(dataFrame['isFraud'], preds)
lr.score(dataFrame[train_cols],dataFrame[test_col])

0.790161624858138

In [ ]:
testpreds = lr.predict(testDF[train_cols])
testpreds

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
#precision_score(testDF['isFraud'], testpreds), recall_score(testDF['isFraud'], testpreds)
lr.score(testDF[train_cols],testDF[test_col])

0.7898294174444802

In [ ]:
#rateFeature(dataFrame.loc[0])

In [ ]:
#features
def rateFeature(row):

  return row['Price']




